In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Đường dẫn tới tệp Excel trong Google Drive
file_path = '/home/nhtrung/main/CLV-Big-Data-Project/data/raw/Online_Retail.xlsx'

# Đọc dữ liệu từ tệp Excel
df = pd.read_excel(file_path)

# Hiển thị một vài hàng đầu tiên của dữ liệu
print(df.head())


: 

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# set to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [3]:
data_null_total = pd.DataFrame(df.isna().sum()).T.rename({0:'total null'})
data_null_total

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
total null,0,0,1454,0,0,0,135080,0


In [4]:
# Drop rows where CustomerID is null
df_cleaned = df.dropna(subset=['CustomerID'])

# Display the number of remaining rows
print(f"Remaining rows after cleaning: {df_cleaned.shape[0]}")

Remaining rows after cleaning: 406829


In [5]:
df_cleaned['CustomerID'].astype(str)


0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 406829, dtype: object

In [6]:
print("Summary Statistics:")
print(df_cleaned.describe())

Summary Statistics:
            Quantity                    InvoiceDate      UnitPrice  \
count  406829.000000                         406829  406829.000000   
mean       12.061303  2011-07-10 16:30:57.879207424       3.460471   
min    -80995.000000            2010-12-01 08:26:00       0.000000   
25%         2.000000            2011-04-06 15:02:00       1.250000   
50%         5.000000            2011-07-31 11:48:00       1.950000   
75%        12.000000            2011-10-20 13:06:00       3.750000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       248.693370                            NaN      69.315162   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


In [7]:
df_cleaned = df_cleaned[df_cleaned['UnitPrice']>0]
df_cleaned = df_cleaned[df_cleaned['Quantity']>0]

In [ ]:
print(f"Remaining rows after cleaning: {df_cleaned.shape[0]}")


Remaining rows after cleaning: 397884


: 

In [ ]:




df = pd.DataFrame(df_cleaned)

# Bước 2: Tính tổng giá trị giao dịch
df['TotalValue'] = df['Quantity'] * df['UnitPrice']


if len(df) > 10000:
    df = df.sample(10000, random_state=42).reset_index(drop=True)

# Bước 3: Nhóm theo CustomerID và tính tổng CLV
clv_data = df.groupby('InvoiceNo').agg({'TotalValue': 'sum'}).reset_index()

: 

In [ ]:
# Bước 4: Tiền xử lý dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(clv_data['TotalValue'].values.reshape(-1, 1))

# Chia dữ liệu thành train (60%), val (20%), test (20%)
train_size = int(len(scaled_data) * 0.6)
val_size = int(len(scaled_data) * 0.2)
test_size = len(scaled_data) - train_size - val_size

train_data = scaled_data[:train_size]
val_data = scaled_data[train_size:train_size + val_size]
test_data = scaled_data[train_size + val_size:]

# Bước 5: Tạo tập dữ liệu cho mô hình với train, validation, và test
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Định nghĩa bước thời gian
time_step = 2

# Tạo tập train, val và test
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape để phù hợp với đầu vào của RNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Bước 6: Xây dựng và huấn luyện mô hình
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình với tập validation
history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_data=(X_val, y_val), verbose=2)

# Bước 8: Dự đoán trên tập test
predicted_clv = model.predict(X_test)
predicted_clv = scaler.inverse_transform(predicted_clv)

# So sánh với y_test (giá trị thật của tập test)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Tính toán các chỉ số đánh giá
mae = np.mean(np.abs(predicted_clv - y_test_actual))
mse = np.mean((predicted_clv - y_test_actual)**2)
rmse = np.sqrt(mse)

print(f'Predicted CLV: {predicted_clv.flatten()}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-8ad3a4a06254>", line 45, in <cell line: 45>
    history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_data=(X_val, y_val), verbose=2)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py", line 318, in fit
    logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 833, in __call__
    result = self._call(*args, **kwds)
  Fi

TypeError: object of type 'NoneType' has no len()

In [ ]:
predicted_clv

array([[28.713133],
       [28.713709],
       [28.714344],
       ...,
       [28.71443 ],
       [28.714855],
       [28.714487]], dtype=float32)

In [ ]:
predicted_clv

array([[28.713133],
       [28.713709],
       [28.714344],
       ...,
       [28.71443 ],
       [28.714855],
       [28.714487]], dtype=float32)

In [ ]:

# Tính toán các chỉ số đánh giá
mae = np.mean(np.abs(predicted_clv - y_test_actual))
mse = np.mean((predicted_clv - y_test_actual)**2)
rmse = np.sqrt(mse)

print(f'Predicted CLV: {predicted_clv.flatten()}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')


Predicted CLV: [28.713133 28.713709 28.714344 ... 28.71443  28.714855 28.714487]
MAE: 27.144547945699582
MSE: 6671.6566694716685
RMSE: 81.68020977857286


In [ ]:
import tensorflow as tf
# Lưu mô hình vào file .h5
model.save("ten_mo_hinh.h5")
